In [1]:
QB_NAME = "Patrick Mahomes"
Year = "2019"

This code is primarily used for analyzing the QBs for my private use in my fantasy football league. 


In [4]:
import os
import yaml
from pathlib import Path

from bs4 import BeautifulSoup
import requests

In [6]:
class QB:
    def __init__(self, name:str):
        self.name: str = name
        self.number: str = ""
        self.position: str = "QB"
        self.team: str = ""
        self.games_played: int = 0
        self.games_started: int = 0
        self.passes_completed: int = 0
        self.passes_attempted: int = 0
        self.pass_completion_percentage: float = 0.0
        self.yards_gained_by_passing: int = 0
        self.passing_touchdowns: int = 0
        self.passing_touchdown_percentage: float = 0.0
        self.interceptions: int = 0
        self.interception_percentage: float = 0.0
        self.longest_completed_pass: int = 0
        self.yards_gained_per_pass_attempt: float = 0.0
        self.yards_gained_per_pass_completion: float = 0.0
        self.qb_rating: float = 0.0
        self.times_sacked: int = 0
        self.yards_lost_due_to_sacks: int = 0
        self.approximate_value: int = 0
        self.year = ""

    def set_stats(self, year:str ) -> None:

        first_letter_lastname = list(self.name.split()[1])[0].upper()
        first_four_lastname = self.name.split()[1][0:4]
        first_two_firstname = self.name.split()[0][0:2]
        self.year = year
        
        if self.is_player_stats_cached():
            print(">> Player stats cached for year, setting from cache")
            self.set_player_stats_from_cache()
            return
        
        print(">> Retrieving data from pro-football-reference.com")

        request_url = "https://www.pro-football-reference.com/players/{}/{}{}00.htm".format(first_letter_lastname,
                                                                                            first_four_lastname,
                                                                                            first_two_firstname)
        try:
            response = requests.get(request_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            stats_for_year = soup.find("tr", {"id": "passing.{}".format(year)})

            self.team  = stats_for_year.find("td", {"data-stat": "team"}).find('a').text
            self.position = stats_for_year.find("td", {"data-stat": "pos"}).text
            self.number = stats_for_year.find("td", {"data-stat": "uniform_number"}).text 
            self.games_played = int(stats_for_year.find("td", {"data-stat": "g"}).text)
            self.games_started = int(stats_for_year.find("td", {"data-stat": "gs"}).text)
            self.passes_completed = int(stats_for_year.find("td", {"data-stat": "pass_cmp"}).text)
            self.passes_attempted = int(stats_for_year.find("td", {"data-stat": "pass_att"}).text)
            self.pass_completion_percentage = float(stats_for_year.find("td", {"data-stat": "pass_cmp_perc"}).text)
            self.yards_gained_by_passing = int(stats_for_year.find("td", {"data-stat": "pass_yds"}).text)
            self.passing_touchdowns = int(stats_for_year.find("td", {"data-stat": "pass_td"}).text)
            self.passing_touchdown_percentage = float(stats_for_year.find("td", {"data-stat": "pass_td_perc"}).text) 
            self.interceptions = int(stats_for_year.find("td", {"data-stat": "pass_int"}).text)
            self.interception_percentage = float(stats_for_year.find("td", {"data-stat": "pass_int_perc"}).text)
            self.longest_completed_pass = int(stats_for_year.find("td", {"data-stat": "pass_long"}).text)
            self.yards_gained_per_pass_attempt = float(stats_for_year.find("td", {"data-stat": "pass_yds_per_att"}).text)
            self.yards_gained_per_pass_completion = float(stats_for_year.find("td", {"data-stat": "pass_yds_per_cmp"}).text)
            self.qb_rating = float(stats_for_year.find("td", {"data-stat": "qbr"}).text)
            self.times_sacked = int(stats_for_year.find("td", {"data-stat": "pass_sacked"}).text)
            self.yards_lost_due_to_sacks = int(stats_for_year.find("td", {"data-stat": "pass_sacked_yds"}).text)
            self.approximate_value = int(stats_for_year.find("td", {"data-stat": "av"}).text)
            #self.save_stats_to_yaml()
        except Exception as e:
            print(e)

    def print_stats(self) -> None:
        print("Year: {}".format(self.year))
        print("Name: {}".format(self.name))
        print("Number: {}".format(self.number))
        print("Position : {}".format(self.position))
        print("Team: {}".format(self.team))
        print("Games played: {}".format(self.games_played))
        print("Games started: {}".format(self.games_started))
        print("Passes completed: {}".format(self.passes_completed))
        print("Passes attempted: {}".format(self.passes_attempted))
        print("Pass completion %: {}".format(self.pass_completion_percentage))
        print("Yards gained by passing: {}".format(self.yards_gained_by_passing))
        print("Passing touchdowns: {}".format(self.passing_touchdowns))
        print("Passing touchdown percentage: {}".format(self.passing_touchdown_percentage))
        print("Interceptions: {}".format(self.interceptions))
        print("Interception percentage: {}".format(self.interception_percentage))
        print("Longest completed pass: {}".format(self.longest_completed_pass))
        print("Yards gained per pass attempted: {}".format(self.yards_gained_per_pass_attempt))
        print("Yards gained per pass completion: {}".format(self.yards_gained_per_pass_completion))
    def save_stats_to_yaml(self) -> None:
        """
        Saves statistical data to a yaml file

        Returns:
            - None
        """
        # if the directory for proper organization doesn't exist, make it
        directory = "./players/QB/{}_{}/".format(self.name.split()[0], self.name.split()[1]) 
        if not os.path.exists(directory):
            os.makedirs(directory)        

        data = {
            "name": self.name,
            "number": self.number,
            "position": self.position,
            "team": self.team,
            "games_played": self.games_played,
            "games_started": self.games_started,
            "passes_completed": self.passes_completed,
            "passes_attempted": self.passes_attempted,
            "pass_completion_perc": self.pass_completion_percentage,
            "yards_gained_by_passing": self.yards_gained_by_passing,
            "passing_touchdowns": self.passing_touchdowns,
            "passing_touchdown_perc": self.passing_touchdown_percentage,
            "interceptions": self.interceptions,
            "interception_perc": self.interception_percentage,
            "longest_completed_pass": self.longest_completed_pass,
            "yards_gained_per_pass_attempt": self.yards_gained_per_pass_attempt,
            "yards_gained_per_pass_completion": self.yards_gained_per_pass_completion,
            
        }

        with open("{}/{}.yaml".format(directory, self.year), "w") as file:
            yaml.dump(data, file, default_flow_style=False)
    
    def is_player_stats_cached(self) -> bool:
     
        player_file = Path("./players/QB/{}_{}/{}.yaml".format(self.name.split()[0], self.name.split()[1], self.year))
        return player_file.is_file()
    
    def set_player_stats_from_cache(self) -> None:
     
        player_file = "./players/QB/{}_{}/{}.yaml".format(self.name.split()[0], self.name.split()[1], self.year)

        with open(player_file, "r") as file:
            try:
                yaml_data = yaml.safe_load(file)
                self.name = yaml_data["name"]
                self.number = yaml_data["number"]
                self.team = yaml_data["team"] 
                self.games_played = yaml_data["games_played"]
                self.games_started = yaml_data["games_started"]
                self.passes_completed = yaml_data["passes_completed"]
                self.passes_attempted = yaml_data["passes_attempted"]
                self.pass_completion_percentage = yaml_data["pass_completion_perc"]
                self.yards_gained_by_passing = yaml_data["yards_gained_by_passing"]
                self.passing_touchdowns = yaml_data["passing_touchdowns"]
                self.passing_touchdown_percentage = yaml_data["passing_touchdown_perc"]
                self.interceptions = yaml_data["interceptions"]
                self.interception_percentage = yaml_data["interception_perc"]
                self.longest_completed_pass = yaml_data["longest_completed_pass"]
                self.yards_gained_per_pass_attempt = yaml_data["yards_gained_per_pass_attempt"]
                self.yards_gained_per_pass_completion = yaml_data["yards_gained_per_pass_completion"]
            except yaml.YAMLError as e:
                print(e)



final = QB(QB_NAME)
final.set_stats(Year)
final.print_stats()


AttributeError: 'QB' object has no attribute 'is_player_stats_cached'